In [1]:
#Word2Vec using Skipgram

import random
import collections
import math
import os
import zipfile
import time
import re

import numpy as np
import tensorflow as tf

from matplotlib import pylab
%matplotlib inline

from six.moves import range
from six.moves.urllib.request import urlretrieve

from tensorflow.contrib.tensorboard.plugins import projector
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

/home/aman/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
dataset_link = 'http://mattmahoney.net/dc/'
zip_file = 'text8.zip'

In [4]:
def data_download(zip_file):
    if not os.path.exists(zip_file):
        zip_file, _ = urlretrieve(dataset_link + zip_file, zip_file)
        print("file downloaded successfully")
    return None    

In [5]:
data_download(zip_file)

file downloaded successfully


In [6]:
#Lets extract the dataset in separate folder

extracte_folder = 'dataset'

if not os.path.isdir(extracte_folder):
    with zipfile.ZipFile(zip_file) as zf:
        zf.extractall(extracte_folder)

In [7]:
with open('dataset/text8') as ft:
    full_text = ft.read()

In [8]:
print(full_text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
def text_processing(ft8_text):
    """Replacing punctuation marks with tokens"""
    ft8_text = ft8_text.lower()
    ft8_text = ft8_text.replace('.', ' <period> ')
    ft8_text = ft8_text.replace(',', ' <comma> ')
    ft8_text = ft8_text.replace('"', ' <quotation> ')
    ft8_text = ft8_text.replace(';', ' <semicolon> ')
    ft8_text = ft8_text.replace('!', ' <exclamation> ')
    ft8_text = ft8_text.replace('?', ' <question> ')
    ft8_text = ft8_text.replace('(', ' <paren_l> ')
    ft8_text = ft8_text.replace(')', ' <paren_r> ')
    ft8_text = ft8_text.replace('--', ' <hyphen> ')
    ft8_text = ft8_text.replace(':', ' <colon> ')
    ft8_text_tokens = ft8_text.split()
    
    return ft8_text_tokens

In [10]:
ft_tokens = text_processing(full_text)

In [11]:
print(ft_tokens)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
word_count = collections.Counter(ft_tokens)

In [13]:
print(word_count)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
shorlisted_words = [w for w in ft_tokens if word_count[w] > 7]

In [15]:
print(shorlisted_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
print(f'Total no. of shorlisted words: {len(shorlisted_words)}')

Total no. of shorlisted words: 16616688


In [18]:
print(f'Unique no. of shorlisted words: {len(set(shorlisted_words))}')

Unique no. of shorlisted words: 53721


In [19]:
def dict_creation(shorlisted_words):
    counts = collections.Counter(shorlisted_words)
    voc = sorted(counts, key=counts.get, reverse=True)
    rev_dict = {i: word for i, word in enumerate(voc)}
    dicti = {word: i for i, word in rev_dict.items()}
    
    return dicti, rev_dict

In [20]:
dicti, rev_dict = dict_creation(shorlisted_words)

In [21]:
word_count = [dicti[word] for word in shorlisted_words]

In [22]:
print(word_count)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
"""Creating the threshold and performing the subsampling"""
thresh = 0.00005
word_counts = collections.Counter(word_count)
total_count = len(word_count)
freqs = {word: count / total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(thresh/freqs[word]) for word in word_counts}
train_words = [word for word in word_count if p_drop[word] < random.random()]